In [1]:
import ngsolve as ngs
from do import solve, solve_2d
from ngsolve.webgui import Draw
import matplotlib.pyplot as plt
from copy import deepcopy
import netgen.occ as occ

levels = 6
deg = 2

######################################################
# from bhdata import BHCurves
# fun_dw  = BHCurves(3)
# fun_w   = fun_dw.Integrate()
# fun_ddw = fun_dw.Differentiate()
######################################################
from bhdata import Brauer
fun_w, fun_dw, fun_ddw  = Brauer()
######################################################

mu0 = 1.256636e-6
nu0 = 1/mu0

linear = "coil_plus|coil_minus"
nonlinear = "stator"

# linear = "stator|coil_plus|coil_minus"
# nonlinear = ""

strom = 1e5
amps = 0.025**2*3.15*strom
print(f'Applying a current of {amps:.2f} Amps')
######################################################

r_coil = 0.025
y_coil = 0.05
r_outer = 0.1

c1 = occ.WorkPlane().Circle(r_outer).Face()
c2 = occ.WorkPlane().MoveTo(0,y_coil).Circle(r_coil).Face()
c3 = occ.WorkPlane().MoveTo(0,-y_coil).Circle(r_coil).Face()

# c1 = occ.WorkPlane().MoveTo(-1,-1).Rectangle(2,2).Face()
# c2 = occ.WorkPlane().MoveTo(-0.75,0).Rectangle(0.5,0.5).Face()
# c3 = occ.WorkPlane().MoveTo(0.25,0).Rectangle(0.5,0.5).Face()

full = occ.Glue([c1,c2,c3])

full.faces[0].name = 'stator'
full.faces[1].name = 'coil_plus'
full.faces[2].name = 'coil_minus'

full.edges[0].name = 'outer'

geoOCC = occ.OCCGeometry(full, dim = 2)
Draw(full)

Applying a current of 196.88 Amps


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': …

BaseWebGuiScene

In [2]:
meshes = []; Bs = []; Bps = []; its = []; errors = []

for i in range(levels):
    ngmesh = geoOCC.GenerateMesh()
    mesh = ngs.Mesh(ngmesh)

    for j in range(i):
        mesh.Refine()
    mesh.Curve(6)

    with ngs.TaskManager(): J = mesh.MaterialCF({'coil_plus': strom, 'coil_minus': -strom, 'stator': 0}, default = 0)
    H1 = ngs.H1(mesh, order = deg, dirichlet = "outer", autoupdate = True)
    L2 = ngs.L2(mesh, dim = 2, order = deg-1)
    # L2 = ngs.HDiv(mesh, order = deg-1)
    A0 = ngs.GridFunction(H1)
    A0, it = solve_2d(H1, A0, mesh, deg, J, fun_w, fun_dw, fun_ddw, linear, nonlinear)
    its.append(it)

    B = ngs.GridFunction(L2, nested = True)
    with ngs.TaskManager(): B.Set(ngs.grad(A0), bonus_intorder = 10)
    Bs.append(B)

    # Draw(B, mesh)

    if i>0:
        L2 = ngs.L2(mesh, dim = 2, order = deg-1)
        # L2 = ngs.HDiv(mesh, order = deg-1)
        Bp = ngs.GridFunction(L2)

        u,v = L2.TnT()
        ir = ngs.IntegrationRule(ngs.fem.ET.TRIG, order = 3*deg)
        with ngs.TaskManager():
            a = ngs.BilinearForm(L2)
            a += ngs.SymbolicBFI(u*v, intrule = ir); a.Assemble()
            f = ngs.LinearForm(L2)
            f += ngs.SymbolicLFI(Bs[i-1]*v, intrule = ir); f.Assemble()
            Bp.vec.data = a.mat.Inverse(freedofs = L2.FreeDofs())*f.vec






        # with ngs.TaskManager(): Bp.Set(Bs[i-1], bonus_intorder = 10)

        Bps.append(Bp)
        # Draw(Bp, mesh)
        with ngs.TaskManager(): error = ngs.Integrate((B-Bp)**2, mesh, order = 3*deg)**(1/2)/\
                                        ngs.Integrate((B)**2, mesh, order = 3*deg)**(1/2)
        print(error)
        errors.append(error)
        

    # ######################################################
    # # Refine mesh, apply prolongation
    # ######################################################

    # ngmesh2 = geoOCC.GenerateMesh()
    # mesh2 = ngs.Mesh(ngmesh2)

    # for j in range(i+1):
    #     mesh2.Refine()
    # mesh2.Curve()

    # V3L2 = ngs.L2(mesh2, dim=3, order = deg-1)
    # Bp = ngs.GridFunction(V3L2)
    # Bp.Set(B)
    # Bps.append(B)


print(its)


# Using 2D mesh with ne= 191 elements and nv= 107 points and  107 DOFs.
 
# Using 2D mesh with ne= 764 elements and nv= 404 points and  404 DOFs.
 
# 0.1701561842247138
# Using 2D mesh with ne= 3056 elements and nv= 1571 points and  1571 DOFs.
 
# 0.1099406013037821
# Using 2D mesh with ne= 12224 elements and nv= 6197 points and  6197 DOFs.
 
# 0.05470195182434864
# Using 2D mesh with ne= 48896 elements and nv= 24617 points and  24617 DOFs.
 
# 0.027303573515624437
# Using 2D mesh with ne= 195584 elements and nv= 98129 points and  98129 DOFs.
 
# 0.013616107336767729
# [8, 9, 9, 9, 10, 10]

Using 2D mesh with ne= 191 elements and nv= 107 points and  404 DOFs.
 
Using 2D mesh with ne= 764 elements and nv= 404 points and  1571 DOFs.
 
0.04977507204956671
Using 2D mesh with ne= 3056 elements and nv= 1571 points and  6197 DOFs.
 
0.01661317335456407
Using 2D mesh with ne= 12224 elements and nv= 6197 points and  24617 DOFs.
 
0.003871957303118123
Using 2D mesh with ne= 48896 elements and nv= 24617 points and  98129 DOFs.
 
0.001001834927167884
Using 2D mesh with ne= 195584 elements and nv= 98129 points and  391841 DOFs.
 
0.00025148443521136977
[8, 13, 9, 10, 11, 11]


In [3]:
import numpy as np
a = np.array(errors)
# a = np.array([0.00011907968926321385,7.813336806776809e-06
# , 3.0480923924668197e-07
# ,1.0782162011697921e-08
# ])
# print(errors)
# print(a[:-1]/a[1:])
print(np.log2(a[:-1]/a[1:]))

# [1.86846328 1.99313412 2.08303115 2.32839828] # gesetz 4
# [1.90055911 2.02087594 2.02998492 2.31871363] # gesetz 3

[1.58309573 2.10119273 1.95041823 1.9941038 ]


In [4]:
with ngs.TaskManager(): Draw(B, mesh)
# with ngs.TaskManager(): Draw(Bp, mesh)
# with ngs.TaskManager(): Draw((ngs.Norm(B-Bp)), mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [5]:
help(ngs.GridFunction.Set)

Help on instancemethod in module ngsolve.comp:

Set(...)
    Set(self: ngsolve.comp.GridFunction, coefficient: ngsolve.fem.CoefficientFunction, VOL_or_BND: ngsolve.comp.VorB = <VorB.VOL: 0>, definedon: object = <ngsolve.ngstd.DummyArgument>, dual: bool = False, use_simd: bool = True, mdcomp: int = 0, definedonelements: Optional[pyngcore.pyngcore.BitArray] = None, bonus_intorder: int = 0) -> None
    
    
    Set values
    
    Parameters:
    
    coefficient : ngsolve.fem.CoefficientFunction
      input CF to set
    
    VOL_or_BND : ngsolve.comp.VorB
      input VOL, BND, BBND, ...
    
    definedon : object
      input definedon region
    
    dual : bool
      If set to true dual shapes are used, otherwise local L2-projection is used.
      Default is False.
    
    use_simd : bool
      If set to false does not use SIMD (for debugging).
    
    mdcomp : int
      .
    
    definedonelements : nullopt
      .
    
    bonus_intorder : int
      Increase numerical integratio

In [6]:
ngs.exp

<function ngsolve.fem.PyCapsule.exp>